# How to use the normal approximations module

In [ ]:
from os import getcwd, path

from datetime import timedelta

from pandas import read_csv, read_json, DataFrame, Series, date_range, concat

from gvar import gvar
from gvar import mean as gv_mean
from gvar import sdev as gv_sdev

from lsqfit import nonlinear_fit

from seaborn import FacetGrid, distplot, despine
from matplotlib.pylab import show as show_plot
from matplotlib.pylab import subplots

from bayes_chime.normal.models import SEIRModel
from bayes_chime.normal import models as m
from bayes_chime.normal.utilities import one_minus_logistic_fcn
from bayes_chime.normal.fitting import fit_norm_to_prior_df, fit_norm_dist_to_ens
from bayes_chime.normal.plotting import (
    plot_prior_fit,
    plot_band,
    plot_gvar,
    plot_posterior_fit,
)

In [ ]:
ROOT = path.dirname(getcwd())
RUN = "2020_04_22_09_07_17"

In [ ]:
OUTPUT = path.join(ROOT, "output", RUN)
DATA = path.join(OUTPUT, "parameters")

## Read in data

In [ ]:
DATA_DF = (
    read_csv(path.join(DATA, "census_ts.csv"), parse_dates=["date"])
    .dropna(how="all", axis=1)
    .fillna(0)
    .set_index("date")
    .astype(int)
)
DATA_DF.head()

## Fit priors

In [ ]:
PRIOR_DF = read_csv(path.join(DATA, f"params.csv"))
PRIOR_DF.head()

In [ ]:
PRIORS = fit_norm_to_prior_df(PRIOR_DF.query("distribution != 'constant'"))
META_PARS = fit_norm_to_prior_df(PRIOR_DF.query("distribution == 'constant'"))

In [ ]:
g = FacetGrid(
    PRIOR_DF.query("distribution != 'constant'"),
    col="param",
    col_wrap=5,
    sharex=False,
    sharey=False,
)
g.map_dataframe(plot_prior_fit)
show_plot(g)
DataFrame(data=PRIORS, index=["val"]).T

## Fit posteriors

The line below which reads in json may take a while. Maybe exporting to `HDF5` might be faster.

In [ ]:
POSTERIOR_DF = read_json(
    path.join(OUTPUT, "output", "chains.json.bz2"), orient="records", lines=True
)
drop_cols = [
    col for col in POSTERIOR_DF.columns if col not in PRIORS and col != "offset"
]
POSTERIOR_DF = POSTERIOR_DF.drop(columns=drop_cols)

The line below may take a while maybe `HDF5` might be a more suiteable format

In [ ]:
POSTERIOR_DF.head()

In [ ]:
thresh = 3.5  # Threshold for identifiying outliers
POSTERIORS = {
    col: fit_norm_dist_to_ens(POSTERIOR_DF[col].values, thresh=thresh) for col in PRIORS
}

stacked = (
    POSTERIOR_DF.T.loc[PRIORS.keys()]
    .T.stack()
    .reset_index()
    .drop(columns=["level_0"])
    .rename(columns={"level_1": "param", 0: "x"})
)
g = FacetGrid(stacked, col="param", col_wrap=5, sharex=False, sharey=False,)
g.map_dataframe(plot_posterior_fit, thresh=thresh)
show_plot(g)
DataFrame(data=POSTERIORS, index=["val"]).T

# Propagate normal posteriors to SEIR and compare to original prediction

In [ ]:
FORECAST_DF = read_csv(
    path.join(OUTPUT, "output", "forecast.csv"), parse_dates=["date"]
).set_index("date")

In [ ]:
seir = SEIRModel()
print(seir.model_parameters)
print(seir.optional_parameters)

In [ ]:
total_infections = (
    META_PARS["n_hosp"] / META_PARS["mkt_share"] / POSTERIORS["hosp_prop"]
)

## Fixed paramters (no distributions)
XX = {
    "dates": FORECAST_DF.index,
    "market_share": META_PARS["mkt_share"],
    "initial_susceptible": META_PARS["region_pop"],
    "initial_infected": 0,
    "initial_recovered": 0,
    "initial_hospitalized": META_PARS["n_hosp"] / META_PARS["mkt_share"],
}
## Variable parameters (distributions)
PP = {
    "initial_exposed": total_infections,
    "incubation_days": POSTERIORS["incubation_days"],
    "beta": POSTERIORS["beta"],
    "recovery_days": POSTERIORS["recovery_days"],
    "nu": POSTERIORS["nu"],
    "hospitalization_probability": POSTERIORS["hosp_prop"],
    "hospital_length_of_stay": POSTERIORS["hosp_LOS"],
}

In [ ]:
def update_parameters(ddate, **kwargs):
    xx = (ddate - kwargs["dates"][0]).days
    ppars = kwargs.copy()
    ppars["beta"] = kwargs["beta"] * one_minus_logistic_fcn(
        xx, L=kwargs["L"], k=kwargs["k"], x0=kwargs["x0"],
    )
    return ppars


OFFSET = POSTERIOR_DF.offset.mean()

PP["L"] = POSTERIORS["logistic_L"]
PP["x0"] = POSTERIORS["logistic_x0"] + OFFSET
PP["k"] = POSTERIORS["logistic_k"]

seir.update_parameters = update_parameters

In [ ]:
%%timeit
seir.propagate_uncertainties(XX, PP)

In [ ]:
FORCAST_DF_NORMAL = seir.propagate_uncertainties(XX, PP)

In [ ]:
fig, axs = subplots(ncols=1, nrows=2, figsize=(6, 9))

gv_kws = dict(color="black", zorder=10, lw=3, z_factor=0.674)
gv_line_kws = {"ls": "--", "label": "Normal posteriors"}
gv_fill_kws = {"alpha": 0.2}

fill_kws = {"alpha": 0.3, "edgecolor": "k", "lw": 2}
line_kws = {"ls": "-", "label": "General posteriors", "lw": 2}

t_range_shifted = FORECAST_DF.index - timedelta(days=OFFSET)

# Census
ax = axs[0]
ax.set_ylabel(f"COVID-19 Hospital Census", fontsize=12, fontweight="bold")
ax.grid(True)

## General posteriors
plot_band(
    x=FORECAST_DF.index,
    y1=FORECAST_DF["Hospitalized Census 25%"],
    ym=FORECAST_DF["Hospitalized Census Median"],
    y2=FORECAST_DF["Hospitalized Census 75%"],
    fill_kws=fill_kws,
    line_kws=line_kws,
    ax=ax,
    zorder=20,
)
## Normal posteriors
plot_gvar(
    x=t_range_shifted,
    y=FORCAST_DF_NORMAL["hospital_census"].values,
    ax=ax,
    **gv_kws,
    line_kws=gv_line_kws,
    fill_kws=gv_fill_kws,
)

ax.legend(bbox_to_anchor=(1.0, 1.0))


# Admits
ax = axs[1]
ax.set_ylabel(f"COVID-19 Hospital Admits", fontsize=12, fontweight="bold")
ax.grid(True)

## General posteriors
plot_band(
    x=FORECAST_DF.index,
    y1=FORECAST_DF["Hospitalized Admits 25%"],
    ym=FORECAST_DF["Hospitalized Admits Median"],
    y2=FORECAST_DF["Hospitalized Admits 75%"],
    fill_kws=fill_kws,
    line_kws=line_kws,
    ax=ax,
    zorder=20,
)

## Normal posteriors
plot_gvar(
    x=t_range_shifted,
    y=FORCAST_DF_NORMAL["hospital_admits"].values,
    ax=ax,
    **gv_kws,
    line_kws=gv_line_kws,
    fill_kws=gv_fill_kws,
)


fig.suptitle(
    "General vs normal posteriors @ 50% C.I.", y=1.02, fontsize=12, fontweight="bold"
)
fig.autofmt_xdate()
fig.tight_layout()

despine()
show_plot(fig)

## Compute normal posteriors given normal priors

In [ ]:
extended_range = date_range(
    DATA_DF.index[0] - timedelta(int(OFFSET)), freq="D", periods=OFFSET
)
tmp = DataFrame(index=extended_range, columns=DATA_DF.columns).fillna(0)
extended_data = concat([tmp, DATA_DF])
extended_data.head()

In [ ]:
total_infections = META_PARS["n_hosp"] / META_PARS["mkt_share"] / PRIORS["hosp_prop"]

## Fixed paramters (no distributions)
xx = {
    "dates": extended_data.index,
    "market_share": META_PARS["mkt_share"],
    "initial_susceptible": META_PARS["region_pop"],
    "initial_infected": 0,
    "initial_recovered": 0,
    "initial_hospitalized": META_PARS["n_hosp"] / META_PARS["mkt_share"],
}
## Variable parameters (distributions)
pp = {
    "initial_exposed": total_infections,
    "incubation_days": PRIORS["incubation_days"],
    "beta": PRIORS["beta"],
    "recovery_days": PRIORS["recovery_days"],
    "nu": PRIORS["nu"],
    "hospitalization_probability": PRIORS["hosp_prop"],
    "hospital_length_of_stay": PRIORS["hosp_LOS"],
}

## It seems like the fit wants to drastically change incubation days and hosp_prob
## The lines below would fix such values to their mean
# pp["incubation_days"] = gvar(pp["incubation_days"].mean, pp["incubation_days"].sdev / 2)
# pp["hospitalization_probability"] = gvar(
#    pp["hospitalization_probability"].mean, pp["hospitalization_probability"].sdev / 5
# )

In [ ]:
pp["L"] = PRIORS["logistic_L"]
pp["x0"] = PRIORS["logistic_x0"] + OFFSET
pp["k"] = PRIORS["logistic_k"]

seir.update_parameters = update_parameters
seir.fit_start_date = "2020-03-06"
seir.fit_columns = "hospital_census"
seir.debug = False

yy = gvar(DATA_DF.hosp.values, DATA_DF.hosp.values * 0.1 + 10)

In [ ]:
seir.check_call(xx, yy, pp)

In [ ]:
%%timeit
fit = nonlinear_fit(data=(xx, yy), prior=pp, fcn=seir.fit_fcn, debug=False)

In [ ]:
fit = nonlinear_fit(data=(xx, yy), prior=pp, fcn=seir.fit_fcn, debug=False)

In [ ]:
x_prediction = xx.copy()
tf = FORECAST_DF.iloc[:100]
x_prediction["dates"] = tf.index
df = seir.propagate_uncertainties(x_prediction, fit.p)
df.index -= timedelta(days=OFFSET)

fig, axs = subplots(ncols=1, nrows=2, figsize=(9, 9), sharex=False)

gv_kws = dict(color="black", zorder=10, lw=3, z_factor=0.674)
gv_line_kws = {"ls": "--", "label": "Normal posteriors"}
gv_fill_kws = {"alpha": 0.2}

fill_kws = {"alpha": 0.3, "edgecolor": "k", "lw": 2}
line_kws = {"ls": "-", "label": "Data", "lw": 2}


# Census
ax = axs[0]
ax.set_ylabel(f"COVID-19 Hospital Census", fontsize=12, fontweight="bold")
ax.grid(True)

## Fit
plot_gvar(
    x=df.index,
    y=df["hospital_census"].values,
    ax=ax,
    **gv_kws,
    line_kws=gv_line_kws,
    fill_kws=gv_fill_kws,
)

## Original
ax.set_ylabel(f"COVID-19 Hospital Census", fontsize=12, fontweight="bold")
plot_band(
    x=tf.index,
    y1=tf["Hospitalized Census 25%"],
    ym=tf["Hospitalized Census Median"],
    y2=tf["Hospitalized Census 75%"],
    fill_kws=fill_kws,
    line_kws=line_kws,
    ax=ax,
    zorder=20,
)

## Data
plot_gvar(
    x=DATA_DF.index,
    y=yy,
    ax=ax,
    z_factor=0.674,
    color="red",
    line_kws={**line_kws, "label": "Data"},
    fill_kws={**fill_kws, "alpha": 0.5, "zorder": 5},
)
ax.legend(loc="upper left")


# Admits
ax = axs[1]
ax.grid(True)

## Prediction
plot_gvar(
    x=df.index,
    y=df["hospital_admits"].values,
    ax=ax,
    **gv_kws,
    line_kws=gv_line_kws,
    fill_kws=gv_fill_kws,
)
ax.grid(True)


## Original
plot_band(
    x=tf.index,
    y1=tf["Hospitalized Admits 25%"],
    ym=tf["Hospitalized Admits Median"],
    y2=tf["Hospitalized Admits 75%"],
    fill_kws=fill_kws,
    line_kws=line_kws,
    ax=ax,
    zorder=20,
)


fig.suptitle(
    "General PDF vs normal PDF @ 50% C.I.", y=1.02, fontsize=12, fontweight="bold"
)
fig.autofmt_xdate()
fig.tight_layout()

despine()
show_plot(fig)

In [ ]:
print(fit)

name_map = {
    "L": "logistic_L",
    "x0": "logistic_x0",
    "k": "logistic_k",
    "hospitalization_probability": "hosp_prop",
    "hospital_length_of_stay": "hosp_LOS",
}

new_posterior = {name_map.get(key, key): val for key, val in fit.p.items()}
new_posterior["logistic_x0"] -= OFFSET
new_prior = {name_map.get(key, key): val for key, val in pp.items()}

comparison = DataFrame(
    [new_prior, new_posterior, POSTERIORS],
    index=["Priors", "PDF from normal approx", "PDF from general dists",],
).T.dropna()

comparison["diff"] = (
    comparison["PDF from normal approx"] - comparison["PDF from general dists"]
)
comparison["z"] = comparison["diff"].apply(lambda x: abs(x.mean) / x.sdev)
comparison.sort_values("z", ascending=False)